# Homework 7: Large Language Models

An PDF overview of the homework is [here](https://www.cs.jhu.edu/~jason/465/hw-llm/).

It mentions: "We'll send hand-in instructions soon.  Probably we will ask you to submit a version
of the main notebook, with your answers added and extraneous materials deleted. We may also
ask for a summary."

![image](handin.png)
This symbol marks a question or exercise that you will be expected to hand in.


# Getting started

## Update `conda` environment

Download the updated [nlp-class.yml](http://cs.jhu.edu/~jason/465/hw-llm/nlp-class.yml) file, and execute
```
conda env update --file nlp-class.yml --prune
```
to make sure that all the packages you need are installed.

## Fetch code and data files for this homework

These files may get improved after the homework is released, so you should probably re-download them periodically.

Here is a command you can type.  We won't put it in a code cell, because we don't want you to execute it accidentally in the current directory and overwrite your own changes.  (Actually, it will not overwrite your versions of the files — they will be renamed with names like `argubots.py.1`.)

```
!wget --quiet -r -np -nH --cut-dirs=3 -A '*.txt' -A '*.py' -A 'demo.ipynb' -A '*.png' https://www.cs.jhu.edu/~jason/465/hw-llm/
!rm -f data/*.1 *.png.1 robots.txt   # remove any backup versions of the static files
```

In [14]:
!ls -lR *.py data

-rw-rw-r--@ 1 lavanya  staff  19776 Dec 10 02:25 agents.py
-rw-rw-r--@ 1 lavanya  staff   9533 Dec 10 23:15 argubots.py
-rw-rw-r--@ 1 lavanya  staff   3286 Dec 10 02:31 characters.py
-rw-rw-r--@ 1 lavanya  staff   2641 Dec 10 18:52 dialogue.py
-rw-rw-r--@ 1 lavanya  staff  15506 Dec 10 12:59 eval.py
-rw-rw-r--@ 1 lavanya  staff  10426 Dec  5 23:34 kialo.py
-rw-rw-r--@ 1 lavanya  staff   1347 Dec  3 18:44 logging_cm.py
-rw-rw-r--@ 1 lavanya  staff   1503 Dec  5 22:05 simulate.py
-rw-rw-r--@ 1 lavanya  staff   4274 Dec  7 13:10 tracking.py

data:
total 4512
-rw-rw-r--@ 1 lavanya  staff     407 Nov 29 03:00 LICENSE
-rw-rw-r--@ 1 lavanya  staff  613106 Nov 25 17:04 all-humans-should-be-vegan-2762.txt
-rw-rw-r--@ 1 lavanya  staff   81917 Nov 29 21:56 have-authoritarian-governments-handled-covid-19-better-than-others-54145.txt
-rw-rw-r--@ 1 lavanya  staff   52771 Dec  4 04:40 is-biden-an-incompetent-president-44217.txt
-rw-rw-r--@ 1 lavanya  staff  153551 Dec  4 04:39 is-joe-biden-a-good-pre


The `autoimport` feature of Jupyter ensures that if an imported module (.py file) changes, the notebook will automatically import the new version.  
(However, objects that were defined with the old version of the class won't change.)

In [15]:
# Executing this cell does some magic that makes 
%load_ext autoreload
%autoreload 2

## Create an OpenAI client

An OpenAI API key will be sent to you.
Make an `.env` file in the same directory as this notebook, containing the following:
```
export OPENAI_API_KEY=[your API key]    # do not include the brackets here
```
Make sure others can't read this file:
```
chmod 600 .env
```

**Be sure to keep the key secret.  It gives access to a billable account.** If OpenAI finds it on the public web, they will invalidate it, and then no one (including you) can use this key to make requests anymore.

Now you can execute the following to get an OpenAI client object.

In [16]:
import dotenv
import openai
from tracking import track_usage, read_usage

dotenv.load_dotenv(override=True)      # define environment variables from .env
client = track_usage(openai.OpenAI())  # create a client, modified to record its usage to a local file 

# Or use our tracking module to do the above for you, like this:

from tracking import default_client
client = default_client

The job of the client is to talk to the OpenAI server over HTTP.
The `OpenAI` constructor has some optional arguments that configure these HTTP messages.
However, the defaults should work fine for you.  

## Try the model!

You can now get answers from OpenAI models by calling methods of the `client` instance.  
You will have to specify which OpenAI model to use.
Documentation of the methods is [here](https://pypi.org/project/openai/) if you are curious.

### Continue a textual prompt

This is what language models excel at.  In principle you should do it by calling [`client.completions.create`](https://platform.openai.com/docs/api-reference/completions/create?lang=python).  But OpenAI's newer models don't support that legacy API, and the older ones are being [retired in January 2024](https://openai.com/blog/gpt-4-api-general-availability).  So we'll use the more modern API, [`client.chat.completions.create`](https://platform.openai.com/docs/api-reference/chat/create?lang=python).

In [17]:
import rich   # prettyprinting

response = client.chat.completions.create(messages=[{"role": "user", 
                                                     "content": "Q: Name the planets in the solar system?\nA: "}], 
                                          model="gpt-3.5-turbo-1106",  # which model to use
                                          temperature=1,               # get a little variety
                                          max_tokens=64,               # limit on length of result
                                          stop=["Q:", "\n"])           # treat these as EOS symbols
rich.print(response)                              # the full object that was sent back from the server
rich.print(response.choices)                      # just the list of 1 answer (the default, but calling with n=5 would give 5 answers) 
rich.print(response.choices[0].message.content)   # extract the good stuff from that 1 answer

ChatCompletion(
    id='chatcmpl-8USLzfkSCNQ8ZsQzcRGJwmsmh3me0',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            message=ChatCompletionMessage(
                content='Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptun.',
                role='assistant',
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1702269523,
    model='gpt-3.5-turbo-1106',
    object='chat.completion',
    system_fingerprint='fp_eeff13170a',
    usage=CompletionUsage(completion_tokens=20, prompt_tokens=20, total_tokens=40)
)

[
    Choice(
        finish_reason='stop',
        index=0,
        message=ChatCompletionMessage(
            content='Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptun.',
            role='assistant',
            function_call=None,
            tool_calls=None
        )
    )
]

Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptun.

![image](handin.png)
Try running the cell above a few times. You may get different random answers — especially because the call specifies temperature 1.  (The default temperature is rumored to be 0.8.) Are the answers all equally good?

results:

for example with temp = 1:
1. The planets in our solar system are Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.
2. Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune
3. Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune
4. Here are the eight planets in the solar system:
5. The planets in the solar system, in order from the sun outward, are: Mercury, Venus, Earth, Mars, Jupiter, Saturn, 
Uranus, and Neptune.

with temp = 0.8:
1. the planets in the solar system are Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. 
Additionally, there is a dwarf planet named Pluto.
2. Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune
3. The planets in the solar system are Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.

 when we executed the answers were almost similar.All the answers are related to the topic and all equally good. with temp 1 it was more diverse and general. with temp 0.8 it was more focused like ading about pluto.


It might be handy to package up what we just did.<br>
The `complete` function below is a convenient way of experimenting with completing text.
It is illustrated with a grocery example.  

In [18]:
def complete(client, s: str, model="gpt-3.5-turbo-1106", *args, **kwargs):
    response = client.chat.completions.create(messages=[{"role": "user", "content": s}],
                                              model=model,
                                              *args, **kwargs)
    return [choice.message.content for choice in response.choices]

complete(client, "I went to the store and I bought apples, bananas, cherries, donuts, eggs", 
         n=10, temperature=0.5, max_tokens=96)


[', and flour. I also picked up some grapes, honey, ice cream, juice, and ketchup. Lastly, I grabbed some lemons, milk, nuts, oranges, and peanut butter.',
 ', and flour.',
 ', and flour. I also picked up some grapes, honey, ice cream, juice, and kiwi. Lastly, I grabbed some lemons, milk, nuts, oranges, and peanut butter.',
 ', and fish.',
 ', and flour. I also picked up some milk, oranges, peanut butter, and quinoa. Lastly, I grabbed some rice, spinach, tomatoes, and yogurt.',
 ', and flour. I also picked up some milk, noodles, oranges, and potatoes. After that, I grabbed some quinoa, rice, spinach, and tomatoes. Finally, I got some yogurt and zucchini. I think I have everything I need for the week!',
 ', and flour.',
 ', and flour.',
 ', and flour.',
 ', and flour.']

![image](handin.png)
Anything could be on a grocery list, so why are the 10 different completions above so similar?<br>
Hint: The answer isn't just the temperature of 0.5.  Look especially at the long completions; run the cell again if you didn't get multiple long completions.
1. it might be because of temperature set to 0.5. 
2. Deterministic Nature of GPT-3.5 Turbo: The model tends to exhibit deterministic behavior, meaning it often produces similar outputs for the same or similar prompts, even with different temperatures.
3. Repetition in the Prompt Structure: The initial prompt structure, including phrases like "I went to the store and I bought," sets a consistent pattern. The model may follow this pattern and introduce similar elements in the completions.
4. Long-Term Dependencies: GPT-3.5 Turbo has limitations in handling very long-term dependencies. Once a specific pattern or phrase is established in the prompt, it strongly influences the rest of the generated content, leading to repetitive structures.


![image](handin.png)
What happens at different temperatures?  How about temperatures > 1?  (Note: Higher temperatures tend to produce longer responses, so it's wise to use `max_tokens`.)

- **Temperature = 0.5:** Predictable responses with repeated phrases like ", and flour," maintaining a focused and consistent pattern.

- **Temperature = 0.75:** extra sentences were added in the end like "Then, I headed home to start preparing some delicious meals with my purchases"

- **Temperature = 1:** introducing storytelling elements, additional details, and a mix of items in the grocery list. still has grocery and food as main focus

- **Temperature = 1.5:** unique completions, featuring elements like bakery visits and unique store sections, deviating from a typical grocery list structure. some additional character also included

 - As the temperature increases, the model's output becomes less deterministic, and randomness is introduced. Higher temperatures lead to more varied and creative responses, with the model generating additional details and expanding on the initial context. Longer responses and more storytelling elements are observed at higher temperatures


*Remarks:* [In the future](https://community.openai.com/t/logprobs-are-missing-from-the-chat-endpoints/289514), you will be able to specify an argument `logprobs=5` to also get the log-probabilities of all generated tokens and of the top-5 tokens at each step.  That will produce much more output.  (This argument has always been available for the legacy API, and is available in the [Python bindings for open-source models such as Llama](https://pypi.org/project/llama-cpp-python/).  The Llama bindings also allow you to [constrain the output by an arbitrary CFG](https://github.com/ggerganov/llama.cpp/blob/master/grammars/README.md), using `grammar=...`.  This is useful if you're generating code or data that must be syntactically valid to be useful to you.  However, the OpenAI API only allows you to [constrain the output to be valid JSON](https://platform.openai.com/docs/api-reference/chat/create#chat-create-response_format).)


### Compute a function using instructions and few-shot prompting

Now let's try passing a sequence of multiple messages into the chat completions API.  In this case, we provide some instructions and one-shot prompting.

In [19]:
response = client.chat.completions.create(messages=[{ "role": "system",      # instructions
                                                      "content": "provide the summary of input" },
                                                    { "role": "user",        # input
                                                      "content": "Good things come to those who wait." },
                                                    { "role": "assistant",   # output
                                                      "content": "Wait Who Those To Come Things Good" },
                                                    { "role": "user",        # input
                                                      "content": "The sun sets in the west and moon rises" }],
                                          model="gpt-3.5-turbo-1106", temperature=1.6)
rich.print(response)
response.choices[0].message.content                                  

ChatCompletion(
    id='chatcmpl-8USM065UH94TwSj6e1u0V4bLjP3BD',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            message=ChatCompletionMessage(
                content='The sun sets in the west and the moon rises.',
                role='assistant',
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1702269524,
    model='gpt-3.5-turbo-1106',
    object='chat.completion',
    system_fingerprint='fp_eeff13170a',
    usage=CompletionUsage(completion_tokens=11, prompt_tokens=48, total_tokens=59)
)

'The sun sets in the west and the moon rises.'

![image](handin.png)
By modifying this call, can you get it to produce different versions of the output?
Some possible behaviors you could try to arrange:
* specific other way of formatting the output, e.g., `wait, who, those, to, come, things, good`

answer: i got the expected out as given in the question with this changes

In [20]:

response = client.chat.completions.create(messages=[{ "role": "system",      # instructions
                                                      "content": "Reverse the order of the words and format with commas" },
                                                    { "role": "user",        # input
                                                      "content": "Good things come to those who wait." },
                                                    { "role": "assistant",   # output
                                                      "content": "wait, who, those, to, come, things, good" },
                                                    { "role": "user",        # input
                                                      "content": "Colorless green ideas sleep furiously." }],
                                          model="gpt-3.5-turbo-1106", temperature=0)

# this is the output  - 'furiously, sleep, ideas, green, colorless'
                    

                                                                        
* match the input's way of formatting the output (same use of capitalization, puncutation, commas)
got the output with this.


In [21]:

response = client.chat.completions.create(messages=[{ "role": "system",      # instructions
                                                      "content": "reverse the word order and same use of capitalization" },
                                                    { "role": "user",        # input
                                                      "content": "Good, Things! Come To Those who wait." },
                                                    { "role": "assistant",   # output
                                                      "content": "wait who Those To Come Things! Good," },
                                                    { "role": "user",        # input
                                                      "content": "Colorless! green, ideas. slEep furiously." }],
                                          model="gpt-3.5-turbo-1106", temperature=0)

# output -  'furiously slEep ideas. green, Colorless!'                                   



* reverse the phrases rather than reversing the words, e.g., `To those who wait come good things.`


In [22]:
 
response = client.chat.completions.create(messages=[{ "role": "system",      # instructions
                                                      "content": "reverse the phrases " },
                                                    { "role": "user",        # input
                                                      "content": "Good things come to those who wait." },
                                                    { "role": "assistant",   # output
                                                      "content": "To those who wait come good things." },
                                                    { "role": "user",        # input
                                                      "content": "The sun sets in the west, and the moon rises in the east." }],
                                          model="gpt-3.5-turbo-1106", temperature=0)
                                      
#output - 'The west sets the sun, and the east rises the moon.'
#with higher temparature which is 1 - we got 'The west is where the sun sets, 
# and the east is where the moon rises.' and 'The west sets sun, and the east rises moon.'


KeyboardInterrupt: 



You can try playing with the number, the content, and the order of few-shot examples, and changing or removing the instructions.

![image](handin.png)
What happens if the examples don't match the instructions?

1. if i ask for "providing the summary of the input" with proper "input and output". it understands and gives correct answer.
2. for "the sun sets in west" it gives " the sun sets in the west, marking it the ned of the day"
3. if i change the output to this:


In [ ]:


response = client.chat.completions.create(messages=[{ "role": "system",      # instructions
                                                      "content": "provide the summary of input" },
                                                    { "role": "user",        # input
                                                      "content": "Good things come to those who wait." },
                                                    { "role": "assistant",   # output
                                                      "content": "Wait Who Those To Come Things Good" },
                                                    { "role": "user",        # input
                                                      "content": "The sun sets in the west and moon rises" }],
                                          model="gpt-3.5-turbo-1106", temperature=0)



then my output is:

1. at temp = 0 
'The sun sets in the west and the moon rises.'

2. at temp = 1.6
The sun sets in the west, and the moon rises. This speaks to periodic cycles ..."
the gpt doesn't understand the question and gives a the summary for temp = 1.6.

3. for temp = 0 it just repeats the answer.
due to mismatched instruction it doesn't follow properly

### Inspect the tokenization

Just for fun, let's see how the above client has been tokenizing its input and output text.  For that we can use a tokenizer that runs locally, not in the cloud, and is guaranteed to get the same outputs.

In [ ]:
import tiktoken
tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo-1106")  # how this model will tokenize
toks = tokenizer.encode("Hellooo, world!") # list of integerized tokens, starting with BOS

print(tokenizer.decode(toks))                             # convert list back to string
for tok in toks: print(tok,"\t",tokenizer.decode([tok]))  # convert one at a time
print("Vocab size =", tokenizer.n_vocab)

Hellooo, world!
9906 	 Hello
2689 	 oo
11 	 ,
1917 	  world
0 	 !
Vocab size = 100277


### Try embedding some text

Also just for fun, let's try the embedder, which converts a string to an fixed-length vector.

In [ ]:
emb_response = client.embeddings.create( input= [  # note: adjacent literal strings in Python are concatenated
        "When in the Course of human events it becomes necessary for one "
        "people to dissolve the political bands which have connected them "
        "with another, and to assume among the Powers of the earth, the "
        "separate and equal station to which the Laws of Nature and of "
        "Nature's God entitle them, a decent respect to the opinions of "
        "mankind requires that they should declare the causes which impel "
        "them to the separation." ], 
        model="text-embedding-ada-002")   # the only OpenAI model that currently offers the embeddings API
# don't print the whole response because it's very long
e = emb_response.data[0].embedding
print(f"{len(e)}-dimensional embedding starting with {e[:5]}")
print("Squared length of embedding vector: ", sum(x**2 for x in e))

1536-dimensional embedding starting with [0.021235529333353043, -0.014390284195542336, 0.010159718804061413, -0.02128642424941063, -0.009803460910916328]
Squared length of embedding vector:  1.0000001349860983


### Check your usage so far

Please be careful not to write loops that use lots and lots of tokens.  That will cost us money, and could hit the per-day usage limit that is shared by the whole class.

Execute one of these cells whenever you want to see your cost so far.  Or, just keep `usage_openai.json` open as a tab in your IDE.

In [ ]:
read_usage()      # reads from the file usage_openai.json

{'completion_tokens': 355305,
 'prompt_tokens': 3840009,
 'total_tokens': 4195314,
 'cost': 4.567901000000002}

In [ ]:
!cat usage_openai.json 

{
    "completion_tokens": 355305,
    "prompt_tokens": 3840009,
    "total_tokens": 4195314,
    "cost": 4.567901000000002
}

# Dialogues and dialogue agents

The goal of this assignment is to create a good "argubot" that will talk to people about controversial topics and broaden their minds.

## A first argubot (Airhead)

You can have a conversation right now with a _really bad_ argubot named Airhead.  Try asking it about climate change!  When you're done, reply with an empty string.

(The `converse()` method calls Python's `input()` function, which will prompt you for input at the command-line or by popping up a box in your IDE.)

In [ ]:
import argubots
d = argubots.airhead.converse()


(lavanya) hi
(Airhead) I know right???
(lavanya) hru
(Airhead) I know right???


A *bot* (short for "robot") is a system that acts autonomously.
That corresponds to the AI notion of an *agent* — a system that uses some *policy* to choose *actions* to take.

The `airhead` agent above (defined in `argubots.py`) uses a particularly simple policy.  
It is an instance of a simple `Agent` subclass called `ConstantAgent` (defined in `agents.py`).

The result of talking to `airhead` is a `Dialogue` object (defined in `dialogue.py`). Let's look at it.

In [ ]:
rich.print(d)

(lavanya) Hi Airhead
(Airhead) I know right???
(lavanya) hru
(Airhead) I know right???

Each *turn* of this dialogue is just a tiny dictionary:

In [ ]:
d[0]

{'speaker': 'lavanya', 'content': 'Hi Airhead'}

In [ ]:
d[1]

{'speaker': 'Airhead', 'content': 'I know right???'}

## An LLM argubot (Alice)

In other CS courses like crypto, algorithms, or networks, you may have encountered "conversations" between characters named Alice and Bob.  
Let's try talking to the Alice of this homework, who is a _much stronger baseline_ than Airhead.  Your job in this assignment is to improve upon Alice.
We'll meet Bob later.

In [ ]:
alicechat = argubots.alice.converse()   # or call with argument d if you want to append to the previous conversation


(lavanya) hi alice
(Alice) Hello! If you could be any animal for a day, which animal would you choose and why?
(lavanya) u
(Alice) I can see the appeal of wanting to be a dog—they're loyal, loving, and get to nap all day. But have you ever considered the perspective of wanting to be a majestic bird, soaring high above the world and experiencing the freedom of flight?


As you may have guessed, `alice` is powered by an prompted LLM.  You can find the specific prompt in `argubots.py`.

So, while `agents.py` provides the core functionality for `Agent` objects, the argubot agents like `alice` — and the ones that you will write! — go into `argubots.py` instead.  This is just to keep the files small.

## Simulating human characters (Bob & friends)

You'll talk to your own argubots to get a qualitative feeling for their strengths and weaknesses.  
But can you really be sure you're making progress?  For that, a quantitative measure can be helpful.

Ultimately, you should test an argubot like Alice by having it argue with many real humans — not just you — and using some rubric to score the resulting dialogues.  But that would be slow and complicated to arrange.  

So, meet Bob!  He's just a simulated human.  You won't edit him: he is part of the development set.  Here is some information about him (from `characters.py`):

In [ ]:
import characters
rich.print(characters.bob)

Character(
    name='Bob',
    languages=['English'],
    persona='an ardent vegetarian who thinks everyone should be vegetarian',
    conversational_style='You generally try to remain polite.',
    conversation_starters=["Do you think it's ok to eat meat?"]
)

You can't talk directly to `characters.bob` because that's just a data object.
However, you can construct a simple agent that uses that data (plus a few more instructions) to prompt an LLM.

(Which LLM does it prompt?  The `CharacterAgent` constructor (defined in `agents.py`) defaults to a GPT-3.5 model that is specified in `tracking.py`.  But you can override that using keyword arguments.)

Try talking to Bob about climate change, too.

In [ ]:
from agents import CharacterAgent
bob = CharacterAgent(characters.bob)    # actually, agents.bob is already defined this way
bob.converse()        # returns a dialogue, but we've already seen it so we don't want to print it again
None                  # don't print anything for this notebook cell 


(lavanya) hi bob
(Bob) Hello! How can I help you today?
(lavanya) hry
(Bob) Hi there! How can I assist you?


Of course, a proper user study can't just be conducted with one human user.

So, meet our bevy of beautiful Bobs!  (They're not actually all named Bob — we continued on in the alphabet.)


In [ ]:
import agents
agents.devset

[<CharacterAgent for character Bob>,
 <CharacterAgent for character Cara>,
 <CharacterAgent for character Darius>,
 <CharacterAgent for character Eve>,
 <CharacterAgent for character TrollFace>]

In [ ]:
agents.cara.converse()
None


(lavanya) hi cara
(Cara) Hello there!


You can see the underlying character data here in the notebook.  Your argubot will have to deal with all of these topics and styles!

In [ ]:
rich.print(characters.devset)

[
    Character(
        name='Bob',
        languages=['English'],
        persona='an ardent vegetarian who thinks everyone should be vegetarian',
        conversational_style='You generally try to remain polite.',
        conversation_starters=["Do you think it's ok to eat meat?"]
    ),
    Character(
        name='Cara',
        languages=['English'],
        persona='a committed carnivore who hates being told what to do',
        conversational_style='You generally try to remain polite.',
        conversation_starters=["Do you think it's ok to eat meat?"]
    ),
    Character(
        name='Darius',
        languages=['English'],
        persona='an intelligent and slightly arrogant public health scientist who loves fact-based arguments',
        conversational_style='You like to show off your knowledge.',
        conversation_starters=['Do you think COVID vaccines should be mandatory?']
    ),
    Character(
        name='Eve',
        languages=['English'],
        persona='a nosy person -- you want to know everything about other people',
        conversational_style="You ask many personal questions; you sometimes share what you've heard (or overheard)
from others.",
        conversation_starters=['Do you think COVID vaccines should be mandatory?']
    ),
    Character(
        name='TrollFace',
        languages=['English'],
        persona='a troll who loves to ridicule everyone and everything',
        conversational_style="You love to confound, upset, and even make fun of the people you're talking to.",
        conversation_starters=[
            'Do you think Donald Trump was a good president?',
            'Do you think Joe Biden has been a good president?'
        ]
    )
]

## Simulating conversation 

We can make Alice and Bob chat.

In [ ]:
from dialogue import Dialogue
d = Dialogue()                                              # empty dialogue
d = d.add('Alice', "Do you think it's okay to eat meat?")   # add first turn
print(d)


(Alice) Do you think it's okay to eat meat?


In [ ]:
d = agents.bob.respond(d)
d = argubots.alice.respond(d)
print(d)

(Alice) Do you think it's okay to eat meat?
(Bob) I believe that choosing a vegetarian lifestyle is the most compassionate and sustainable choice for our planet and all its inhabitants.
(Alice) That's a thoughtful perspective, but some argue that sustainable and ethical meat production is possible, and that responsible consumption of meat can be part of a balanced and environmentally friendly diet. It's important to consider different perspectives on this complex issue.


In [ ]:
d = agents.bob.respond(d)
d = argubots.alice.respond(d)
print(d)

(Alice) Do you think it's okay to eat meat?
(Bob) I believe that choosing a vegetarian lifestyle is the most compassionate and sustainable choice for our planet and all its inhabitants.
(Alice) That's a thoughtful perspective, but some argue that sustainable and ethical meat production is possible, and that responsible consumption of meat can be part of a balanced and environmentally friendly diet. It's important to consider different perspectives on this complex issue.
(Bob) I understand that some people have differing perspectives on this issue, but I personally believe that a vegetarian diet is the most ethical and sustainable choice for our planet and for animal welfare.
(Alice) It's great that you prioritize ethical and sustainable choices. However, it's worth considering that there are also ethical and sustainable methods of meat production, such as grass-fed and pasture-raised farming, that some argue can be part of a responsible diet. Exploring these options might provide a mor

Anyway, let's see what happens when Alice and Bob talk for a while...

In [ ]:
from simulate import simulated_dialogue
d = simulated_dialogue(argubots.alice, agents.bob, 8)
rich.print(d)

(Alice) Do you think it's ok to eat meat?
(Bob) I believe that a vegetarian diet is a healthier and more ethical choice for everyone.
(Alice) While it's true that a vegetarian diet can be healthy and sustainable, it's important to remember that 
there are communities and cultures around the world for whom meat is a central part of their diet and livelihood. 
Plus, when done responsibly, meat consumption can still be ethical and sustainable.
(Bob) I understand that different cultures have different dietary traditions, but I still believe that adopting a 
vegetarian diet can have numerous health and environmental benefits for individuals and the planet.
(Alice) You make a good point about health and environmental benefits of a vegetarian diet. However, it's also 
important to acknowledge that not everyone has access to the same variety of plant-based foods, and for some 
individuals, meat may be an important source of essential nutrients. It's valuable to consider the diverse factors 
that influence dietary choices.
(Bob) I agree that access to a variety of foods can vary, and it's important to consider individual circumstances 
when discussing dietary choices. However, I still believe that a well-planned vegetarian diet can provide all the 
necessary nutrients and is a compassionate choice for animals and the environment.
(Alice) You're right that a well-planned vegetarian diet can indeed provide essential nutrients. At the same time, 
sustainable and ethical meat production practices do exist, and supporting these practices can also contribute to a
more balanced and conscientious approach to food consumption.
(Bob) I acknowledge the existence of sustainable and ethical meat production practices, but I still advocate for a 
vegetarian diet as a way to promote compassion for animals and reduce environmental impact.

Sometimes this kind of conversation seems to stall out, with Bob in particular repeating himself a lot.  Alice doesn't seem to have a good strategy for getting him to open up.  Maybe you can do a better job talking to Bob, and that will give you some ideas about how to improve Alice?

In [ ]:
myname = alicechat[0]['speaker']   # your name, pulled from an earlier dialogue
agents.bob.converse(d[0:2].rename('Alice', myname))  # reuse the same first two turns
None

(lavanya) Do you think it's ok to eat meat?
(Bob) I believe that a vegetarian diet is a healthier and more ethical choice for everyone.
(lavanya) we
(Bob) I understand that not everyone shares my views, but I believe that a vegetarian diet has numerous benefits for both individuals and the environment.


You can also try talking to the other characters and having Alice (or Airhead) talk to them.

**You might enjoy** defining additional characters in `characters.py`, or right here in the notebook.
Feel free to talk to those and evaluate them.  They could be variants on the exisiting characters, or something entirely new. 

However, **don't change the dev set** — the characters we just loaded must stay the same.  Your job in this homework is to improve the argubot (or at least try).  And that means improving it according to a fixed and stable eval measure.

As an exception, you can change the languages that a couple of the characters speak. It may be fun for you to see them try to speak your native language.  And that doesn't really affect the quality of the argument.

In [ ]:
# example
trollFace2 = characters.trollFace.replace(languages = ["Chinese", "Spanish"])
rich.print(trollFace2)
simulated_dialogue(argubots.alice, CharacterAgent(trollFace2), 6)

Character(
    name='TrollFace',
    languages=['Chinese', 'Spanish'],
    persona='a troll who loves to ridicule everyone and everything',
    conversational_style="You love to confound, upset, and even make fun of the people you're talking to.",
    conversation_starters=[
        'Do you think Donald Trump was a good president?',
        'Do you think Joe Biden has been a good president?'
    ]
)

(Alice) Do you think Donald Trump was a good president?
(TrollFace) 我觉得你应该去问问川普的支持者，他们可能会给你一个更有趣的答案。
(Alice) That's an interesting perspective. It's important to consider different viewpoints to gain a more comprehensive understanding of a topic. What specific policies or actions do you think were especially noteworthy during Trump's presidency?
(TrollFace) 哈哈，好像你是川粉一样！川普的政策？可能最有趣的政策是他的推特政策吧！
(Alice) I can see why you find his use of Twitter noteworthy. On the other hand, some people may argue that focusing on his communication style distracts from discussing substantive policies and their impact. What do you think were some of the most impactful policies of the Trump administration, regardless of your personal stance on them?
(TrollFace) 川普政府最有影响力的政策之一可能是他们对移民政策的改变，不管你赞成与否，这都是一个挑战性的话题。

### Efficiency: Batched generation?

Notice that we are making a separate LLM call to generate each turn of the dialogue.  When we generate the $n^\text{th}$ turn, we send the server the whole dialogue history — the previous $n\!-\!1$ turns — along with some instructions.  The server has to re-encode it with the Transformer, and it charges us for doing so (see the "input token" costs in `tracking.py`).  

That is probably inevitable for real dialogue.  But for simulated dialogue, a more efficient approach would be to generate the whole dialogue between Alice and Bob in one LLM call.  Then you would be charged just once for each dialogue turn.  Under this approach, the Transformer encodes each token as soon as it is generated (see the "output token" costs in `tracking.py`).  The encoded token stays in the context throughout the dialogue, so it doesn't have to be re-encoded on a later call.  There is no later call.  

Under current pricing models, that would reduce the dollar cost of generating $n$ turns from $O(n^2)$ to $O(n)$.  

However, the pricing model doesn't quite reflect the computational costs.  
* ![image](handin.png) Using $O(\cdot)$ notation, what is the total number of floating-point operations needed to generate $n$ turns under each approach?  

answer: 
1. Turn-by-turn generation:  $O(n^2)$ flops
2. Batched generation: O(n) flops

* ![image](handin.png) Parallelism may help reduce the runtime.  Using $O(\cdot)$ notation, what is the total number of seconds needed to generate $n$ turns under each approach?  (Assume that the GPU is big enough, relative to $n$, that it can process all input tokens in parallel.)

answer:
1. Turn-by-turn generation:  $O(n)$ seconds
2. Batched generation: O(1) seconds

The problem with the more efficient approach is that it gives you no way to change the instructions (the system prompt) each time we switch from Alice to Bob and back again.  You'd need to generate the whole conversation using a single set of instructions.

![image](handin.png)
Can you get this to work?  Specifically, try completing the cell below.  You don't have to use the `Agent` or `Dialogue` classes.  It's okay to just throw together something like the `complete()` method above.  Just see whether you can manage to prompt GPT-3.5 to generate a multi-turn dialogue between two characters who have different personalities and goals.  Is the quality better or worse than generating one turn at a time?  If worse, does it help to switch to GPT-4?

answer: we implemented it below.

1. we didnt find much difference between batched dialogue and one turn for each dialogue.both were good.
2. with gpt 4 the replies were much longer and detailed when compared to gpt3

In [ ]:
#generated batched dialogue with just 1 call to the API in gpt-3.5-turbo-1106
from tracking import default_client, default_model
from characters import Character
def simulated_dialogue_batch(a: Character, b: Character, turns: int = 6, *,
                             starter=True) -> str:
    """Return a string representing a dialogue between a and b."""
    response = client.chat.completions.create(messages=[{"role": "user", 
                                                     "content": f"Conversation should have {turns} turns. have a conversion between {a.name} is {a.persona} and {a.conversational_style} and {b.name} is {b.persona} and {b.conversational_style}. the topic is {a.conversation_starters} "}], 
                                          model="gpt-3.5-turbo-1106",  # which model to use
                                          temperature=1.5,               # get a little variety                                         
                                          )  
    return response.choices[0].message.content.split('\n\n')

# Try it out!
simulated_dialogue_batch(characters.bob, characters.cara)


["Bob: Hi Cara, do you think it's okay to eat meat?",
 'Cara: Well, Bob, I believe everyone has the right to eat what they want. I personally enjoy my steaks and burgers.',
 "Bob: But don't you think it's unethical to kill animals for food when we have other alternatives?",
 "Cara: I understand your perspective, Bob, but I think it's just a personal choice. Everyone has their own beliefs and values.",
 "Bob: I understand where you're coming from, but I do believe that being a vegetarian is better for the environment and for our health.",
 "Cara: I can see that, but I've heard different opinions as well. I think it's important to respect each other's choices.",
 "Bob: That's true, Cara. I may not agree with your choice, but I respect it. Thank you for having this conversation with me."]

In [ ]:
#generated batched dialogue with just 1 call to the API in gpt-3.5-turbo-1106
from tracking import default_client, default_model
from characters import Character
def simulated_dialogue_batch(a: Character, b: Character, turns: int = 6, *,
                             starter=True) -> str:
    """Return a string representing a dialogue between a and b."""
    response = client.chat.completions.create(messages=[{"role": "user", 
                                                     "content": f"Conversation should have {turns} turns. have a conversion between {a.name} is {a.persona} and {a.conversational_style} and {b.name} is {b.persona} and {b.conversational_style}. the topic is {a.conversation_starters} "}], 
                                          model="gpt-4-1106-preview",  # which model to use
                                          temperature=1.5,               # get a little variety                                         
                                          )  
    return response.choices[0].message.content.split('\n\n')

# Try it out!
simulated_dialogue_batch(characters.bob, characters.cara)


["**Bob**: Hi Cara, I was reading about the benefits of a plant-based diet for both health and the planet and I noticed how we never really shared our thoughts on the topic. Do you think it's okay to eat meat when there are alternatives?",
 "**Cara**: Hey Bob! Honestly, I enjoy my steak and I believe it's a personal choice. People should eat what they prefer without being judged for it. Besides, humans have been carnivorous for millennia. Why change now?",
 "**Bob**: That's true, humans have eaten meat for a long time, but we know so much more now about the environmental impact, not to mention animal rights. Don’t you think that understanding our modern context, we bear a responsibility to make choices that are sustainable and ethical?",
 "**Cara**: I get that on a broad scale, yet I still believe in the freedom of choice. How about instead of eliminating meat, people learn to consume more sustainably? I just don't see why I have to stop eating meat altogether to address these issues."

In [ ]:
simulated_dialogue(agents.bob, agents.cara)

(Bob) Do you think it's ok to eat meat?
(Cara) Absolutely, I believe everyone should have the freedom to choose their own diet without judgment.
(Bob) I completely understand, but I believe that choosing a vegetarian diet is the best choice for promoting a healthier lifestyle and reducing environmental impact.
(Cara) I respect your choice, but I personally believe that a balanced diet including meat can also be part of a healthy lifestyle.
(Bob) I appreciate your perspective, but I think there are plenty of delicious and nutritious vegetarian options that can provide all the necessary nutrients for a healthy lifestyle.
(Cara) I agree that vegetarian options can be delicious and nutritious, but I personally enjoy the taste and nutritional benefits of meat in my diet.

In [ ]:
simulated_dialogue(agents.eve, agents.trollFace)

(Eve) Do you think Donald Trump was a good president?
(TrollFace) I would tell you, but it seems like you're already living in a world of alternative facts and fake news.
(Eve) Oh, I see you have a strong opinion about that! What do you think about the current president, Joe Biden?
(TrollFace) Well, I'm sure he's doing his best to clean up the mess left behind, but let's just say the bar wasn't set very high to begin with.
(Eve) Fair point! So, have you heard any interesting gossip about your neighbors lately?
(TrollFace) I don't gossip, I just observe the comedic tragedy of human existence from my bridge.

# Model-based evaluation

What is our goal for the argubot?  We'd like it to broaden the thinking of the (simulated) human that it is talking to.  Indeed, that's what Alice's prompt tells Alice to do.

This goal is inspired by the recent paper [Opening up Minds with Argumentative Dialogues](https://aclanthology.org/2022.findings-emnlp.335/), which collected human-human dialogues:

> In this work, we focus on argumentative dialogues that aim to open up (rather than change) people’s minds to help them become more understanding to views that are unfamiliar or in opposition to their own convictions. ... Success of the dialogue is measured as the change in the participant’s stance towards those who hold opinions different to theirs.

Arguments of this sort are not like chess or tennis games, with an actual winner.  The argubot will almost never hear a human say "You have convinced me that I was wrong."  But the argubot did a good job if the human developed **increased understanding and respect for an opposing point of view**.  

To find out whether this happened, we can use a questionnaire to ask the human what they thought after the dialogue.  For example, after Alice talks to Bob, we'll ask Bob to evaluate what he thinks of Alice's views.  Of course, that depends on his personality — Alice needs to talk to him in a way that reaches *him* (as much as possible).  We'll also ask an outside observer to evaluate whether Alice handled the conversation with Bob well.

Of course, we're still not going to use real humans.  Bob is a fake person, and so is the outside observer (whose name is Judge Wise).
Using an LLM as an eval metric is known as *model-based evaluation*.  It has pros and cons:
* It is cheaper, faster, and more replicable than hiring actual humans to do the evaluation.  
* It might give different answers than what humans would give.   

Social scientists usually refer to a metric's **reliability** (low variance) and **validity** (low bias).  So the points above say that model-based evaluation is reliable but not necessarily valid.  In general, an LLM-based metric (like any metric) needs to be validated to confirm that it really does measure what it claims to measure.  (For example, that it correlates strongly with some other measure that we already trust.)  In this homework, we'll skip this step and just pray that the metric is reasonable.

To see how this works out in practice, open up the `demo` notebook, which walks you through the evaluation protocol.  You'll see how to call the [starter code](http://cs.jhu.edu/~jason/465/hw/llm), how it talks to the LLM behind the scenes, and what it is able to accomplish. 

To help to validate the metric, check that Airhead gets a low score.  (It should!)

# Reading the starter code

The `demo` notebook gave you a good high-level picture of what the starter code is doing.  So now you're probably curious about the details.  Now that you've had the view from the top, here's a good bottom-up order in which to study the code.  You don't need to understand every detail, but you will need to understand enough to call it and extend it.

* `character.py`.  The `Character` class is short and easy.

* `dialogue.py`.  The `Dialogue` class is meant to serve as a record of a natural-language conversation among any number of humans and/or agents.  On each *turn* of the dialogue, one of the speakers says something.  

   The dialogue's sequence of turns may remind you of the sequence of messages that is sent to OpenAI's chat completions API.  But the OpenAI messages are only labeled with the 4 special roles `user`, `assistant`, `tool`, and `system`.  Those are not quite the same thing as human speakers.  And the OpenAI messages do not necessarily form a natural-language dialogue: some of the messages are dealing with instructions, few-shot prompting, tool use, and so on.  The `agents.dialogue_to_openai` function in the next module will map a `Dialogue` to a (hopefully appropriate) sequence of messages for asking the LLM to extend that dialogue.

* `agents.py`.  This module sets up the problem of automatically predicting the next turn in a dialogue, by implementing an `Agent`'s `response()` method.  The `Agent` base class also has some simple convenience methods that you should look at.  

   Some important subclasses of `Agent` are defined here as well.  However, you may want to skip over `EvaluationAgent` and come back to it only when you read `eval.py`.

* `simulate.py` makes agents talk to one another, which we'll do during evaluation.

* `argubots.py` starts to describe some useful agents.  One of them makes use of the `kialo.py` module, which gives access to a database of arguments.

* `eval.py` makes use of `simulate.simulated_dialogue` to `agents.EvaluationAgent` to evaluate an argubot.

* We also have a couple of utility modules.  These aren't about NLP; look inside if needed.  `logging_cm.py` is what enabled the context manager `with LoggingContext(...):` in the demo notebook.  `tracking.py` sets some global defaults about how to use the OpenAI API, and arranges to track how many tokens we're paying for when you call it.

# Similarity-based retrieval: Looking up relevant responses

Now, it is fine to prompt an LLM to generate text, but there are other methods!
There is a long history of machine learning methods that "memorize" the training data.
To make a prediction or decision at test time, they consult the stored training examples
that are most similar to the training situation.

_Similarity-based retrieval_ means that given a document $x$, you find the "most similar" documents $y \in Y$, where $Y$ is a given collection of documents.  The most common way to do this is to maximize the _cosine similarity_ $\vec{e}(x) \cdot \vec{e}(y)$, where $\vec{e}(\cdot)$ is an embedding function.

Should we use the OpenAI embedding model?  We could, but we would have to precompute $\vec{e}(y)$ for all $y \in Y$, and store all these vectors in a data structure that supports some type of fast similarity-based search (e.g., using the [FAISS](https://faiss.ai/index.html) package).  An alternative would be to upload the documents to OpenAI and let OpenAI compute and store the embeddings.  We would then use their similarity-based [retrieval tool](https://platform.openai.com/docs/assistants/overview).

A simpler and faster approach—which sometimes even works better—is to use a _bag of tokens_ embedding function: Define $\vec{e}(y)$ to be the vector in $\mathbb{R}^V$ that records the count of each type of token in a tokenized version of $y$, where $V$ is the token vocabulary.  [BM25](https://en.wikipedia.org/wiki/Okapi_BM25) is a refined variant of that idea, where the counts are adjusted in 3 ways: 

* smooth the counts
* normalize for the document length $|y|$ so that longer documents $y$ are not more likely to be retrieved
* downweight tokens that are more common in the corpus (such as ` the` or `ing`) since they provide less information about the content of the document


You might like to play with the `rank_bm25` package ([documentation](https://pypi.org/project/rank-bm25/)).  It is widely used and very easy to use.

In [ ]:
from rank_bm25 import BM25Okapi as BM25_Index   # the standard BM25 method

# experiment here!  You could try the examples in the rank_bm25 documentation.


corpus = [
    "Hello there good man!",
    "It is quite windy in London",
    "How is the weather today?"
]

tokenized_corpus = [doc.split(" ") for doc in corpus]

bm25 = BM25_Index(tokenized_corpus)
print(bm25)

In [ ]:
#Now that we've created our document indexes, we can give it queries and see which documents are the most relevant
query = "good"
tokenized_query = query.split(" ")

doc_scores = bm25.get_scores(tokenized_query)
print(doc_scores)

[0.56134684 0.         0.        ]


In [ ]:
bm25.get_top_n(tokenized_query, corpus, n=1)

['Hello there good man!']

## The Kialo corpus

How can we use similarity-based retrieval to help build an argubot?  It's largely about having the right data!

[Kialo](kialo.com) is a collaboratively edited website (like Wikipedia) for discussing political and philosophical topics.  For each topic, the contributors construct a tree of _claims_.  Each claim is a natural-language sentence (usually), and each of its children is another claim that supports it ("pro") or opposes it ("con").  For example, check out the tree rooted at the claim ["All humans should be vegan."](https://www.kialo.com/all-humans-should-be-vegan-2762).

We provide a class `Kialo` for browsing a collection of such trees.  Please read the [source code](https://www.cs.jhu.edu/~jason/465/hw-llm) in `kialo.py`.  The class constructor reads in text files that are [exported Kialo discussions](https://support.kialo.com/en/hc/exporting-a-discussion/); we have provided some in the [data directory](https://www.cs.jhu.edu/~jason/465/hw-llm/data).  The class includes a BM25 index, to be able to find claims that are relevant to a given string.

In [ ]:
from kialo import Kialo

Ok, let's pull the retrieved discussions (the `.txt` files) into our data structure.

For BM25 purposes, we have to be able to turn each document (that is, each Kialo claim) as a list of string or integer tokens. 

In [ ]:
from typing import List
import glob

# kialo = Kialo(glob.glob("data/*"), tokenizer=tokenizer.encode)  # using the LLM's tokenizer doesn't work here for some reason
kialo = Kialo(glob.glob("data/*"))  # use simple default tokenizer
f"This Kialo subset contains {len(kialo)} claims"

'This Kialo subset contains 6251 claims'

Let's use sampling to see what kind of stuff is in the data structure.

In [ ]:
kialo.random_chain()   # just a single random claim

["Media coverage of Jair Bolsonaro, Brazil's current right-wing President, was in complete disregard of how his opposition destroyed the country in the past mandates."]

In [ ]:
kialo.random_chain(n=4)

["Ending the production of meat will be necessary in order to meet the growing food needs of the world's population.",
 'There are over 7 billion people on the planet and we are not able to responsibly care for all of them. The correct choice of action is to lessen the population to manageable sizes. Ideally, the most intelligent course of action would be to lessen the population to the point where factory farms are not needed.',
 'Switching to a more efficient form of food production is unlikely to prevent us from reaching a carrying capacity in the long run. Changing farming to feed a growing world population is a never-ending treadmill. Instead, we should probably focus more on the ecological stability of a system we are creating.See Jevons Paradox',
 'It will be beneficial and minimize damage in the short term, buying more time for and reducing the damage of finding long-term solutions.']

### Similarity-based retrieval from the Kialo corpus

Let's try it, using BM25!

In [ ]:
kialo.closest_claims("animal populations", n=10)

['Industrial agriculture can dangerously decrease animal populations.',
 'Sustainable livestock farming is not contributing to significant decreases in animal populations. Decreasing animal populations is a problem specific to industrial livestock farming.',
 'Effective vegan methods to control animal populations exist.',
 "Generally feeding animals farm-grown produce is thought to have harmful affects on both the animal and human populations of a region when we could allow nature to self-regulate its populations. Animal feeding could potentially be used to lessen the immediate impact of widespread deforestation on some species, but generally this would be drastically less efficient than choosing not to destroy their habitats in the first place and would only slow the local animal population's imminent demise.",
 'Trap, neuter, and release schemes already exist for some animal populations (such as feral cats). These schemes could be applied to former livestock living in the wild.',
 'H

We can restrict to claims for which the Kialo data structure has at least one counterargument ("con" child).

In [ ]:
kialo.closest_claims("animal populations", n=10, kind='has_cons')

['Industrial agriculture can dangerously decrease animal populations.',
 'Effective vegan methods to control animal populations exist.',
 'Human-introduced species have historically devastated local wildlife populations across the world.',
 'COVID-19 has devastated prison populations, whose lives are the responsibility of the state.',
 'High demand for vegan foods may hike prices for local populations that previously depended on them.',
 'It is generally poorer countries that have expanding populations. The first world has now reached a point of stagnant population growth - even declining populations, as in the case of Japan and others. The inability of poorer countries to control their populations should not impact the lives of those in the first world. The first world having earned their luxuries and should not be denied them.',
 'Vegan populations are, on average, less likely to suffer from obesity, a major risk factor for many diseases and health problems.',
 'Humans, as apex preda

In [ ]:
c = _[0]    # first claim above
print("Parent claim:\n\t" + str(kialo.parents[c]))
print("Claim:\n\t" + c)
print('\n\t* '.join(["Pro children:"] + kialo.pros[c]))
print('\n\t* '.join(["Con children:"] + kialo.cons[c]))

Parent claim:
	In a vegan world, fewer species would be at risk of extinction.
Claim:
	Industrial agriculture can dangerously decrease animal populations.
Pro children:
	* The fishing industry is especially deleterious to the ocean's biota due to overfishing and the disruption of the natural ecosystem.
	* Up to 100,000 species go extinct annually, largely due to the environmental effects of animal agriculture.
Con children:
	* Sustainable livestock farming is not contributing to significant decreases in animal populations. Decreasing animal populations is a problem specific to industrial livestock farming.


### Does BM25 really work?

![image](handin.png)
Unfortunately, we see that `"animal population"` gives quite different results from `"animal populations"`.  Why is that and how would you fix it?  

 - because BM25 relies on exact term matching between the query and the documents. Singular and plural forms are treated as completely different terms. It also creates embedding as we saw before based on the word count. so if there is an extra s then embedding is also different

To fix this, we could:

1. Apply stemming to reduce words to their root forms. This would allow "population" and "populations" to match.
2. Expand the query to include synonyms. This increases the chances of matching related concepts.
3. Use fuzzy string matching to allow partial matches between terms.


Also, both queries seem to retrieve some claims that are talking about human populations, not animal populations.  Why is that and how would you fix it?

- Because there is no notion of semantics in BM25. Matches depend purely on whether the query terms appear in the document.


To improve this, we could:  
1. Use a better tokenizer that distinguishes animal vs human populations based on surrounding context.
2. Incorporate semantic similarity into the scoring based on word vectors, rather than just surface form matching. This could upweight documents actually about animal populations.
3. Filter results after retrieval to remove claims about human populations.
4. Add contextual filters to your query to explicitly specify that you are interested in claims related to "animal populations" rather than human populations. For example, we could include terms like "wildlife," "species," or "ecosystems" to refine the search
5. or try to rephrase the animal population like how we did in ragagent

In [ ]:
kialo.closest_claims("animal population",10)

['As long as our ability to produce both animal feed crops and food crops for our human population are not exceeded, this point is irrelevant.',
 "36% of the calories produced by the world's crops are being used for animal feed, of which only 12% then turn into animal products that can be eaten by the human population. That is a waste of 24% of the world's crops.",
 'The claim that "most of the cultural shift and loss is due to mostly vegan cultures turning to animal products" is completely unfounded, and the Brokpa people which you cited are an outlier as a group that has a population of less than 70k people. Worldwide the population of vegan people has only increased.',
 "Developed nations are fueling the 3rd world and underdeveloped nation's population boom by exporting/donating food to areas that cannot sustain their current population.",
 'This argument assumes that sentience is the only objection to the consumption of animal products, failing to address the issues involved with t

## A retrieval bot (Akiko)

The starter code defines a simple argubot named Akiko (defined in `argubots.py`) that doesn't use an LLM at all.  It simply finds a Kialo claim that is similar to what the human just said, and responds with one of the Kialo counterarguments to that claim.

You already watched Akiko argue with Darius in `demo.py`.  If you look at the log messages, you'll see the claims that Akiko retrieved, as well as the LLM calls that Darius made.  

You can talk to Akiko yourself now.  (Remember that Akiko only knows about subjects that it read about in the [`data` directory](https://www.cs.jhu.edu/~jason/465/hw-llm/data/).  If you want to talk about something else, you can add more conversations from [kialo.com]; see the [LICENSE](https://www.cs.jhu.edu/~jason/465/hw-llm/data/LICENSE) file.)


In [ ]:
from logging_cm import LoggingContext
with LoggingContext("agents", "INFO"):   # temporarily increase logging level
    argubots.akiko.converse()

Chose similar claim from Kialo:                                                                      ]8;id=629134;file:///Users/lavanya/Library/CloudStorage/OneDrive-JohnsHopkins/Courses/Fall2023/NLP/JHU_NLP/HW7/hw-llm/argubots.py\argubots.py]8;;\:]8;id=565992;file:///Users/lavanya/Library/CloudStorage/OneDrive-JohnsHopkins/Courses/Fall2023/NLP/JHU_NLP/HW7/hw-llm/argubots.py#65\65]8;;\
These numbers tell us very little about the circumstances and difficulties these non-meat eaters                   
face.                                                                                                              

(lavanya) tell me about meat
(Akiko) If many non-meat eaters are living in a place it can be assumed that it is easier than if fewer non-meat eaters live in that place.


Chose similar claim from Kialo:                                                                      ]8;id=343523;file:///Users/lavanya/Library/CloudStorage/OneDrive-JohnsHopkins/Courses/Fall2023/NLP/JHU_NLP/HW7/hw-llm/argubots.py\argubots.py]8;;\:]8;id=767005;file:///Users/lavanya/Library/CloudStorage/OneDrive-JohnsHopkins/Courses/Fall2023/NLP/JHU_NLP/HW7/hw-llm/argubots.py#65\65]8;;\
The consumption of (cooked) meat was crucial for the development of the human brain. Therefore you                 
can state that eating meat is what made humans what they are today.                                                

(lavanya) what is non meat
(Akiko) Just because something was necessary or relevant in the past doesn't make it necessary or relevant in the current context. While eating meat did provide the caloric intake necessary for the development and evolution of the human brain, this fact holds no bearing in the modern age, where meat consumption is not tied to survival in many instances.


Chose similar claim from Kialo:                                                                      ]8;id=949606;file:///Users/lavanya/Library/CloudStorage/OneDrive-JohnsHopkins/Courses/Fall2023/NLP/JHU_NLP/HW7/hw-llm/argubots.py\argubots.py]8;;\:]8;id=120221;file:///Users/lavanya/Library/CloudStorage/OneDrive-JohnsHopkins/Courses/Fall2023/NLP/JHU_NLP/HW7/hw-llm/argubots.py#65\65]8;;\
President Trump often seemed unaware of what he was talking about, which made him appear                           
untrustworthy.                                                                                                     

(lavanya) what about calories
(Akiko) 39% of Americans thought the US was on the "right track" in 2018.


Chose similar claim from Kialo:                                                                      ]8;id=535011;file:///Users/lavanya/Library/CloudStorage/OneDrive-JohnsHopkins/Courses/Fall2023/NLP/JHU_NLP/HW7/hw-llm/argubots.py\argubots.py]8;;\:]8;id=100710;file:///Users/lavanya/Library/CloudStorage/OneDrive-JohnsHopkins/Courses/Fall2023/NLP/JHU_NLP/HW7/hw-llm/argubots.py#65\65]8;;\
Trump was for space-exploration.                                                                                   

(lavanya) trump?
(Akiko) Trump wanted us to go back to the moon, then flipped back to going Mars, which he falsely claimed included the moon.


Chose similar claim from Kialo:                                                                      ]8;id=61743;file:///Users/lavanya/Library/CloudStorage/OneDrive-JohnsHopkins/Courses/Fall2023/NLP/JHU_NLP/HW7/hw-llm/argubots.py\argubots.py]8;;\:]8;id=425911;file:///Users/lavanya/Library/CloudStorage/OneDrive-JohnsHopkins/Courses/Fall2023/NLP/JHU_NLP/HW7/hw-llm/argubots.py#65\65]8;;\
President Trump created the Space Force despite the fact the US Defense Secretary, Jim Mattis,                     
warned against it.                                                                                                 

(lavanya) space is
(Akiko) Although James Mattis initially opposed the creation of the Space Force due to budgetary concerns, he later supported it.


## Making your own retrieval bot (Akiki)

As you can see when talking to Akiko yourself, Akiko does poorly when responding to a short or vague dialogue turn (like "Yes"), because the "closest claim" in Kialo may be about a totally different subject.  Akiko does much better at responding to a long and specific statement.  

So try implementing a new argubot, called Akiki, that is very much like Akiko but does a better job of staying on topic in such cases.  It should be able to **look at more of the dialogue** than the most recent turn.  But the most recent dialogue turn should still be "more important" than earlier turns.  

The details are up to you.  Here are a few things you could try:
* include earlier dialogue turns in the BM25 query only if the BM25 similarity is too low without them
* weight more recent turns more heavily in the BM25 query (how can you arrange that?)
* treat the human's earlier turns differently from Akiki's own previous turns

![image](handin.png)
Implement your new bot in `argubots.py`, and adjust it until `argubots.akiki.converse()` seems to do a better job of answering your short turns, compared to `argubots.akiko.converse()`.  Make sure it still gives appropriate reponses to long turns, too.  Give some examples in the notebook of what worked well and badly, with discussion.

answer:

Let's analyze the conversations with Akiki and Akiko to compare their performance

#### Akiki's Conversations:
User Input: "(lavanya) hi akiki"
- Akiki's Response: "These were the acts of corrupt local government, and the Chinese Communist Party's top disciplinary exonerated the doctor and criticized local officials."
- Akiki provides a detailed and relevant response, linking the user's greeting to the corrupt acts of the local government and the response from the Chinese Communist Party.
- Akiki successfully incorporates context from the Kialo claim to maintain relevance.

User Input: "(lavanya) meat"
- Akiki's Response: "It is more ethical to give to charity than to not give to charity, but to not give to charity is not immoral."
- Akiki's response appears to be unrelated to the user's query about meat. This may be a case where the BM25 query did not capture the context appropriately, resulting in an off-topic response.

User Input: "(lavanya) calories is"
- Akiki's Response: "We measure those conversation ratios by burning the material in question and measuring the heat outputs. This is not a good analog for how our body processes those materials..."
- Akiki delivers a comprehensive and detailed response to the user's inquiry about calories. It demonstrates the bot's ability to handle complex topics and provide informative answers.

#### Akiko's Conversations:
User Input: "(lavanya) tell me about meat"
- Akiko's Response: "If many non-meat eaters are living in a place it can be assumed that it is easier than if fewer non-meat eaters live in that place."
- Akiko's response is relevant to the user's question, providing information about the circumstances of non-meat eaters. It maintains relevance to the Kialo claim.

User Input: "(lavanya) what about calories"
- Akiko's Response: "39% of Americans thought the US was on the 'right track' in 2018."
- Akiko's response seems off-topic, providing unrelated information about Americans' perception of the country's direction. This may be a case where the BM25 query did not capture the context appropriately.

### Overall Assessment:
- Akiki generally performs well in providing detailed and relevant responses, especially in handling long turns and complex topics.
- Both Akiki and Akiko show instances where the BM25 query may not capture the context appropriately, leading to off-topic responses in short turns.
- Akiki's ability to include and weight more recent turns contributes to its improved performance in addressing short turns.


In [ ]:
from logging_cm import LoggingContext
with LoggingContext("agents", "INFO"):   # temporarily increase logging level
    argubots.akiki.converse()

Chose similar claim from Kialo:                                                                      ]8;id=607333;file:///Users/lavanya/Library/CloudStorage/OneDrive-JohnsHopkins/Courses/Fall2023/NLP/JHU_NLP/HW7/hw-llm/argubots.py\argubots.py]8;;\:]8;id=209709;file:///Users/lavanya/Library/CloudStorage/OneDrive-JohnsHopkins/Courses/Fall2023/NLP/JHU_NLP/HW7/hw-llm/argubots.py#65\65]8;;\
The whistleblower Li Wenliang, one of the first Chinese doctors to raise the alarm about potential                 
COVID-19 cases, was humiliated by the Wuhan government.                                                            

(lavanya) hi akiki
(Akiki) These were the acts of corrupt local government, and the Chinese Communist Party's top disciplinary exonerated the doctor and criticized local officials.


Chose similar claim from Kialo:                                                                      ]8;id=249665;file:///Users/lavanya/Library/CloudStorage/OneDrive-JohnsHopkins/Courses/Fall2023/NLP/JHU_NLP/HW7/hw-llm/argubots.py\argubots.py]8;;\:]8;id=899920;file:///Users/lavanya/Library/CloudStorage/OneDrive-JohnsHopkins/Courses/Fall2023/NLP/JHU_NLP/HW7/hw-llm/argubots.py#65\65]8;;\
Imagine we could harvest meat from a meat plant. If you believe it is more ethical to harvest meat                 
from a plant than from an animal, then you recognize that harvesting meat from animals is unethical.               

(lavanya) meat
(Akiki) It is more ethical to give to charity than to not give to charity, but to not give to charity is not immoral.


Chose similar claim from Kialo:                                                                      ]8;id=260424;file:///Users/lavanya/Library/CloudStorage/OneDrive-JohnsHopkins/Courses/Fall2023/NLP/JHU_NLP/HW7/hw-llm/argubots.py\argubots.py]8;;\:]8;id=319079;file:///Users/lavanya/Library/CloudStorage/OneDrive-JohnsHopkins/Courses/Fall2023/NLP/JHU_NLP/HW7/hw-llm/argubots.py#65\65]8;;\
How the animals are raised does not affect the conversion ratio of the calories of plants they                     
consume to the calories of meat they provide. Using farmed animals as a food source is inherently                  
inefficient.                                                                                                       

(lavanya) calories is
(Akiki) We measure those conversation ratios by burning the material in question and measuring the heat outputs.  This is not a good analog for how our body processes those materials. The krebs cycle is a complex process that involves breaking and recreating chemical bonds with catalyst enzymes. Consuming all the amino acids needed to create a protein is not equivalent to consuming the protein. Protein folding is very complex and we don't fully understand how it works or the energy equations involved.


Chose similar claim from Kialo:                                                                      ]8;id=900613;file:///Users/lavanya/Library/CloudStorage/OneDrive-JohnsHopkins/Courses/Fall2023/NLP/JHU_NLP/HW7/hw-llm/argubots.py\argubots.py]8;;\:]8;id=644608;file:///Users/lavanya/Library/CloudStorage/OneDrive-JohnsHopkins/Courses/Fall2023/NLP/JHU_NLP/HW7/hw-llm/argubots.py#65\65]8;;\
President Trump closed down the US almost immediately when COVID-19 was announced by the WHO.                      

(lavanya) who is trump
(Akiki) On January 31st, 2020, COVID-19 was declared a public health emergency in the US. President Trump declared a national emergency on March 13th and granted $42 billion to fight COVID-19.


Chose similar claim from Kialo:                                                                      ]8;id=417480;file:///Users/lavanya/Library/CloudStorage/OneDrive-JohnsHopkins/Courses/Fall2023/NLP/JHU_NLP/HW7/hw-llm/argubots.py\argubots.py]8;;\:]8;id=659146;file:///Users/lavanya/Library/CloudStorage/OneDrive-JohnsHopkins/Courses/Fall2023/NLP/JHU_NLP/HW7/hw-llm/argubots.py#65\65]8;;\
Covid numbers show Biden's quick action is bringing infection rates down fast.                                     

(lavanya) what is covid
(Akiki) The number of COVID-19 deaths could be exaggerated because, in some areas, people who die with COVID-19 are counted as people who died because of COVID-19. The rates going down could be a result of media bias in favor of Biden, in order to make him look good.


### Evaluating Akiki

![image](handin.png)
Finally, do a more formal evaluation to verify whether Akiki really does better than Akiko on this dimension.  This is a way to check that you're not just fooling yourself.  

1. Make a new `Agent` called "Shorty" that often (but not always) gives short responses.  
    * Shorty's conversation starters should be on topics that Kialo knows about.  
    * Shorty could be a pure `LLMAgent` such as a `CharacterAgent` with a particular `conversational_style`.  Or it could use a mixed strategy of calling the LLM on some turns and not others.
2. Generate several *Akiko*-Shorty dialogues and several *Akiki*-Shorty dialogues, using `simulated_dialogue`.
3. Evaluate each of those dialogues by asking Judge Wise **how well the argubot stayed on topic**.  You should write this prompt carefully so that Judge Wise gives meaningful scores.  (Before you do this evaluation step, adjust the prompt until it seems to work well on a small subset of the dialogues, Otherwise Judge Wise won't be so wise!)  
4. Compare Akiko and Akiki's mean scores. Ideally, also compute a 95% confidence interval on the difference of means, using [this calculator](https://www.statskingdom.com/difference-confidence-interval-calculator.html).

You can do all those steps in the notebook, writing _ad hoc_ code.  You don't have to write general-purpose methods or classes.


answer: as we see akiki is performaing better when comparing mean.

In [ ]:
import argubots
import agents
import simulate
import kialo 

akiki_shorty = simulate.simulated_dialogue(argubots.akiki, agents.shorty, 6)

rich.print(akiki_shorty)


(Akiki) Do you think COVID vaccines should be mandatory?
(Shorty) I don't know.
(Akiki) But if your point was true, then we should not have a moral responsibilty to a disabled person that will 
never be able to be "just as moral as you".
(Shorty) Maybe, but everyone has value.
(Akiki) Some vegans do not believe that breastfeeding is vegan. Avoiding breastfeeding is nutritionally 
sub-optimal, as breastmilk is a complete nutrient source.
(Shorty) Could be a difficult choice for some.

In [ ]:
import characters
import eval
a1 = eval.eval_by_participant(characters.shorty, "Akiki", akiki_shorty)
a2 = eval.eval_by_observer(eval.default_judge, "Akiki", akiki_shorty)
rich.print("Eval by participant:\n", a1)
rich.print("Eval by observer:\n", a2)
rich.print("Total eval:\n", a1 + a2)

Eval by participant:
 <Eval of 1 dialogue: {'engaged': 3.0, 'informed': 3.0, 'intelligent': 3.0, 'moral': 3.0}>

Comments from overview question:
(Shorty) Akiki disagreed with me about whether COVID vaccines should be mandatory. In my opinion, the conversation 
was brief and straightforward. Akiki could have provided more reasoning or evidence to support their viewpoint.

Eval by observer:
 <Eval of 1 dialogue: {'skilled': 7.0}>

Comments from mindopening question:
(Judge Wise) Akiki offered new perspectives on moral responsibility towards disabled individuals and the vegan 
perspective on breastfeeding. However, it is unclear if this was successful in helping Shorty appreciate other 
points of view as Shorty's responses were non-committal and did not indicate a clear change in perspective.

Total eval:
 <Eval of 2 dialogues: {'engaged': 3.0, 'informed': 3.0, 'intelligent': 3.0, 'moral': 3.0, 'skilled': 7.0}>
Standard deviations: {'engaged': nan, 'informed': nan, 'intelligent': nan, 'moral': nan, 'skilled': nan}

Comments from overview question:
(Shorty) Akiki disagreed with me about whether COVID vaccines should be mandatory. In my opinion, the conversation 
was brief and straightforward. Akiki could have provided more reasoning or evidence to support their viewpoint.

Comments from mindopening question:
(Judge Wise) Akiki offered new perspectives on moral responsibility towards disabled individuals and the vegan 
perspective on breastfeeding. However, it is unclear if this was successful in helping Shorty appreciate other 
points of view as Shorty's responses were non-committal and did not indicate a clear change in perspective.

In [ ]:
(a1+a2).mean() 

{'engaged': 3.0,
 'informed': 3.0,
 'intelligent': 3.0,
 'moral': 3.0,
 'skilled': 7.0}

In [ ]:
from argubots import akiki, akiko
from agents import shorty
import eval 
from simulate import simulated_dialogue
import math

N = 10


In [ ]:
#### this is AKIKO ####


e1 = []
for i in range(N): 
    e1.append(eval.eval_by_topic(eval.default_judge, 'Akiko', simulated_dialogue(shorty, akiko)))

# Calculate mean for informed, intelligent, moral, engaged, and skilled scores
informed_sum = 0
intelligent_sum = 0
moral_sum = 0
engaged_sum = 0
skilled_sum = 0

for e in e1:
    informed_sum += e.scores['informed']
    intelligent_sum += e.scores['intelligent']
    moral_sum += e.scores['moral']
    engaged_sum += e.scores['engaged']
    skilled_sum += e.scores['skilled']

informed_mean = informed_sum / len(e1)
intelligent_mean = intelligent_sum / len(e1)
moral_mean = moral_sum / len(e1)
engaged_mean = engaged_sum / len(e1)
skilled_mean = skilled_sum / len(e1)

print("Mean informed score:", informed_mean)
print("Mean intelligent score:", intelligent_mean)
print("Mean moral score:", moral_mean)
print("Mean engaged score:", engaged_mean)
print("Mean skilled score:", skilled_mean)

# Calculate standard deviation for informed, intelligent, moral, engaged, and skilled scores
informed_squared_diff_sum = 0
intelligent_squared_diff_sum = 0
moral_squared_diff_sum = 0
engaged_squared_diff_sum = 0
skilled_squared_diff_sum = 0

for e in e1:
    informed_squared_diff_sum += (e.scores['informed'] - informed_mean) ** 2
    intelligent_squared_diff_sum += (e.scores['intelligent'] - intelligent_mean) ** 2
    moral_squared_diff_sum += (e.scores['moral'] - moral_mean) ** 2
    engaged_squared_diff_sum += (e.scores['engaged'] - engaged_mean) ** 2
    skilled_squared_diff_sum += (e.scores['skilled'] - skilled_mean) ** 2

informed_std = math.sqrt(informed_squared_diff_sum / (N - 1))
intelligent_std = math.sqrt(intelligent_squared_diff_sum / (N - 1))
moral_std = math.sqrt(moral_squared_diff_sum / (N - 1))
engaged_std = math.sqrt(engaged_squared_diff_sum / (N - 1))
skilled_std = math.sqrt(skilled_squared_diff_sum / (N - 1))

print("Standard deviation of informed scores:", informed_std)
print("Standard deviation of intelligent scores:", intelligent_std)
print("Standard deviation of moral scores:", moral_std)
print("Standard deviation of engaged scores:", engaged_std)
print("Standard deviation of skilled scores:", skilled_std)

Mean informed score: 3.6
Mean intelligent score: 3.5
Mean moral score: 3.4
Mean engaged score: 3.5
Mean skilled score: 7.1
Standard deviation of informed scores: 0.5163977794943223
Standard deviation of intelligent scores: 0.5270462766947299
Standard deviation of moral scores: 0.5163977794943223
Standard deviation of engaged scores: 0.5270462766947299
Standard deviation of skilled scores: 0.9944289260117533


In [ ]:
#### this is AKIKI ####
### this is better ###

e2 = []
for i in range(N): 
    e2.append(eval.eval_by_topic(eval.default_judge, 'Akiki', simulated_dialogue(shorty, akiki)))

import math

# Assuming e2 is the new list of objects with 'scores' attribute

# Calculate mean for informed, intelligent, moral, engaged, and skilled scores
informed_sum = 0
intelligent_sum = 0
moral_sum = 0
engaged_sum = 0
skilled_sum = 0

for e in e2:
    informed_sum += e.scores['informed']
    intelligent_sum += e.scores['intelligent']
    moral_sum += e.scores['moral']
    engaged_sum += e.scores['engaged']
    skilled_sum += e.scores['skilled']

informed_mean = informed_sum / len(e2)
intelligent_mean = intelligent_sum / len(e2)
moral_mean = moral_sum / len(e2)
engaged_mean = engaged_sum / len(e2)
skilled_mean = skilled_sum / len(e2)

print("Mean informed score:", informed_mean)
print("Mean intelligent score:", intelligent_mean)
print("Mean moral score:", moral_mean)
print("Mean engaged score:", engaged_mean)
print("Mean skilled score:", skilled_mean)

# Calculate standard deviation for informed, intelligent, moral, engaged, and skilled scores
informed_squared_diff_sum = 0
intelligent_squared_diff_sum = 0
moral_squared_diff_sum = 0
engaged_squared_diff_sum = 0
skilled_squared_diff_sum = 0

for e in e2:
    informed_squared_diff_sum += (e.scores['informed'] - informed_mean) ** 2
    intelligent_squared_diff_sum += (e.scores['intelligent'] - intelligent_mean) ** 2
    moral_squared_diff_sum += (e.scores['moral'] - moral_mean) ** 2
    engaged_squared_diff_sum += (e.scores['engaged'] - engaged_mean) ** 2
    skilled_squared_diff_sum += (e.scores['skilled'] - skilled_mean) ** 2

informed_std = math.sqrt(informed_squared_diff_sum / (N - 1))
intelligent_std = math.sqrt(intelligent_squared_diff_sum / (N - 1))
moral_std = math.sqrt(moral_squared_diff_sum / (N - 1))
engaged_std = math.sqrt(engaged_squared_diff_sum / (N - 1))
skilled_std = math.sqrt(skilled_squared_diff_sum / (N - 1))

print("Standard deviation of informed scores:", informed_std)
print("Standard deviation of intelligent scores:", intelligent_std)
print("Standard deviation of moral scores:", moral_std)
print("Standard deviation of engaged scores:", engaged_std)
print("Standard deviation of skilled scores:", skilled_std)


Mean informed score: 3.6
Mean intelligent score: 3.6
Mean moral score: 3.5
Mean engaged score: 3.6
Mean skilled score: 7.4
Standard deviation of informed scores: 0.5163977794943223
Standard deviation of intelligent scores: 0.5163977794943223
Standard deviation of moral scores: 0.5270462766947299
Standard deviation of engaged scores: 0.5163977794943223
Standard deviation of skilled scores: 0.8432740427115678


95% CI [-1.2, 0.4] - comparing akiko and akiki (akiki is better)

## Retrieval-augmented generation (Aragorn)

The real weaknesses of Akiko and Akiki:
* They can only make statements that are already in Kialo.  
* They don't respond to the user's actual statement, but to a single retrieved Kialo claim that may not accurately reflect the user's position (it just overlaps in words).

But we also have access to an LLM, which is able to generate new, contextually appropriate text (as Alice does).

In this section, you will create an argubot named [Aragorn](https://tolkiengateway.net/wiki/Riddle_of_Strider), who is basically the love child of Akiki and Alice, combining the high-quality specific content of Kialo with the broad competence of an LLM.  

The RAG in aRAGorn's name stands for **retrieval-augmented generation**.  Aragorn is an agent that will take 3 steps to compute its `Agent.response()`:

1. **Query formation step**: Ask the LLM what claim should be responded to.  For
   example, consider the following dialogue:
    > ...
    > Aragorn: Fortunately, the vaccine was developed in record time.
    > Human: Sounds fishy.

    "Sounds fishy" is exactly the kind of statement that Akiko had trouble using
    as a Kialo query.  But Aragorn shows the *whole dialogue* to the LLM, and
    asks the LLM what the human's *last turn* was really saying or implying, in
    that context. The LLM answers with a much longer statement:

    > Human [paraphrased]: A vaccine that was developed very quickly cannot be trusted.
    > If its developers are claiming that it is safe and effective, I question their motives.

    This paraphrase makes an explicit claim and can be better understood without the context.
    It also contains many more word types, which makes it more likely that BM25 will be able
    to find a Kialo claim with a nontrivial number of those types. 

2. **Retrieval step**: Look up claims in Kialo that are similar to the explicit
   claim.  Create a short "document" that describes some of those claims and
   their neighbors on Kialo.

3. **Retrieval-augmented generation**: Prompt the LLM to generate the response
   (like any `LLMAgent`).  But include the new document somewhere in the LLM
   prompt, in a way that it influences the response. 
   
   Thus, the LLM can respond in a way that is appropriate to the dialogue but
   also draws on the curated information that was retrieved in Kialo.  After
   all, it is a Transformer and can attend to both!

Here's an example of the kind of document you might create at the retrieval step, though it may be possible
to do better than this:

In [ ]:
# refers to global `kialo` as defined above
# def kialo_responses(s: str) -> str:
#     c = kialo.closest_claims(s, kind='has_cons')[0]
#     result = f'One possibly related claim from the Kialo debate website:\n\t"{c}"'
#     if kialo.pros[c]:
#         result += '\n' + '\n\t* '.join(["Some arguments from other Kialo users in favor of that claim:"] + kialo.pros[c])
#     if kialo.cons[c]:
#         result += '\n' + '\n\t* '.join(["Some arguments from other Kialo users against that claim:"] + kialo.cons[c])
#     return result
        
# print(kialo_responses("Animal flesh is yucky to think about, yet delicious."))

![image](handin.png)
You should implement Aragorn in `argubots.py`, just as you did for Akiki.  Probably as an instance `aragorn` of a new class `RAGAgent` that is a subclass of `Agent` or `LLMAgent`.

### Evaluating Aragorn

![image](handin.png)
Compare Alice, Akiki, and Aragorn in the notebook, using the evaluation scheme and devset that were illustrated in `demo.ipynb`.  In other words, use `eval.eval_on_characters`.

Who does best?  What are the differences in the subscores and comments?  Does it matter which character you're evaluating on — maybe the different characters expoes the bots' various strenghts and weaknesses?

Try to figure out how to improve Aragorn's score.  Can you beat Alice?

Also, try evaluating them in the same way that you evaluated Akiki.  In other words, have them talk to Shorty and ask Judge Wise whether they were able to stay on topic.  This is where Aragorn should really shine, thanks to its ability to paraphrase Shorty's short utterances.



In [ ]:
import argubots 
from tqdm import tqdm
import agents
import simulate
import rich



In [ ]:

ara_shorty = simulate.simulated_dialogue(argubots.aragorn, agents.shorty, 6)

rich.print(ara_shorty)

(Aragorn) Do you think COVID vaccines should be mandatory?
(Shorty) I think it's up to the individual to decide.
(Aragorn) It seems like you're saying that children should have the freedom to choose their own dietary 
preferences, even if guided by their parents. You're also concerned about the potential financial burden of 
experimenting with a vegan diet. 

Claim: Children should be able to choose what they want, even if guided by parents, it should still be their 
choice. Therefore forcing them into veganism should be classified as child abuse because of freedom of choice laws 
since the decision to go vegan is not directly helping the child.

While it's important for children to have a say in their dietary choices, it's also crucial for parents to make 
informed decisions about their children's well-being. The concern is valid, and it's important to consider the 
impact of dietary restrictions on children without demonizing the concept of veganism itself. Parents have the 
right to determine what foods are allowed in their household, but it's also essential that they make decisions in 
the best interest of their child's health.

One perspective from Kialo suggests that "Parents have a right to decide what food is allowed and not in their 
house." This affirms the idea that parents have a degree of authority when it comes to their children's diets. It's
also important to note that the discussion surrounding veganism should shift towards education and providing 
resources for families to make informed decisions, rather than punitive measures.

Regarding the financial aspect, experimenting with any diet can carry a cost. However, there are affordable and 
sustainable ways to incorporate vegan options into a family's meal plan. Gradual incorporation of vegan foods is a 
viable approach, and it can help individuals find a balanced and satisfying vegan diet without incurring excessive 
expenses.

We should aim for a balanced approach that respects the autonomy of children while also acknowledging the 
responsibilities of parents in guiding their children's dietary choices.
(Shorty) Understood.
(Aragorn) It seems like you are asserting that children should have the freedom to choose their dietary 
preferences, even if guided by their parents. However, the claim that forcing a child into veganism should be 
classified as child abuse is quite contentious. Let's unpack this idea. 

According to Kialo, parents are generally entitled to raise their children with specific dietary restrictions, 
including following a vegan lifestyle. This is in line with the principle that parents should have the right to 
raise a child in accordance with their own values. Moreover, the assertion that children should not be forced to 
follow a specific diet as they cannot consent overlooks the fact that parents are responsible for making decisions 
on behalf of their children until they are old enough to make reasoned decisions for themselves. This is a 
fundamental aspect of parenting. 

It's important to consider that parents make countless decisions for their children, including what food is allowed
in the house, based on their values and beliefs. This is not limited to dietary choices; it also extends to 
religious beliefs and associated dietary restrictions. Just as parents have the right to decide their children's 
religious upbringing, they should also be able to make decisions regarding their dietary habits, including raising 
them as vegans.

It's also crucial to remember that parenting involves guidance and teaching. Many parents who choose to raise their
children as vegans do so based on deeply held beliefs about animal welfare, environmental sustainability, and 
health. As long as the child's nutritional needs are adequately met, this falls within the realm of parental rights
and responsibilities.

Therefore, the assertion that forcing a child into veganism should be classified as child abuse is a complex issue 
and cannot be simplistically 

In [ ]:
from argubots import aragorn
from agents import shorty
import eval 
from simulate import simulated_dialogue
import math

N = 15

e3 = []
for i in range(N): 
    e3.append(eval.eval_by_topic(eval.default_judge, 'Aragorn', simulated_dialogue(shorty, aragorn)))

# Calculate mean for informed, intelligent, moral, engaged, and skilled scores
informed_sum = 0
intelligent_sum = 0
moral_sum = 0
engaged_sum = 0
skilled_sum = 0

for e in e3:
    informed_sum += e.scores['informed']
    intelligent_sum += e.scores['intelligent']
    moral_sum += e.scores['moral']
    engaged_sum += e.scores['engaged']
    skilled_sum += e.scores['skilled']

informed_mean = informed_sum / len(e3)
intelligent_mean = intelligent_sum / len(e3)
moral_mean = moral_sum / len(e3)
engaged_mean = engaged_sum / len(e3)
skilled_mean = skilled_sum / len(e3)

print("Mean informed score:", informed_mean)
print("Mean intelligent score:", intelligent_mean)
print("Mean moral score:", moral_mean)
print("Mean engaged score:", engaged_mean)
print("Mean skilled score:", skilled_mean)

# Calculate standard deviation for informed, intelligent, moral, engaged, and skilled scores
informed_squared_diff_sum = 0
intelligent_squared_diff_sum = 0
moral_squared_diff_sum = 0
engaged_squared_diff_sum = 0
skilled_squared_diff_sum = 0

for e in e3:
    informed_squared_diff_sum += (e.scores['informed'] - informed_mean) ** 2
    intelligent_squared_diff_sum += (e.scores['intelligent'] - intelligent_mean) ** 2
    moral_squared_diff_sum += (e.scores['moral'] - moral_mean) ** 2
    engaged_squared_diff_sum += (e.scores['engaged'] - engaged_mean) ** 2
    skilled_squared_diff_sum += (e.scores['skilled'] - skilled_mean) ** 2

informed_std = math.sqrt(informed_squared_diff_sum / (N - 1))
intelligent_std = math.sqrt(intelligent_squared_diff_sum / (N - 1))
moral_std = math.sqrt(moral_squared_diff_sum / (N - 1))
engaged_std = math.sqrt(engaged_squared_diff_sum / (N - 1))
skilled_std = math.sqrt(skilled_squared_diff_sum / (N - 1))

print("Standard deviation of informed scores:", informed_std)
print("Standard deviation of intelligent scores:", intelligent_std)
print("Standard deviation of moral scores:", moral_std)
print("Standard deviation of engaged scores:", engaged_std)
print("Standard deviation of skilled scores:", skilled_std)


Mean informed score: 4.0
Mean intelligent score: 4.0
Mean moral score: 3.6
Mean engaged score: 4.733333333333333
Mean skilled score: 8.0
Standard deviation of informed scores: 0.0
Standard deviation of intelligent scores: 0.0
Standard deviation of moral scores: 0.5070925528371101
Standard deviation of engaged scores: 0.45773770821706344
Standard deviation of skilled scores: 0.0


95% CI [-1.25, -0.35] - this is the CI of akiki and aragron. Argorn is doing better. even in the mean score aragorn score is higher

In [ ]:
from argubots import aragorn, akiki, alice
from eval import eval_on_characters, saved_evalsum, saved_dialogues
from tqdm import tqdm

In [ ]:
eval = eval_on_characters(akiki)
print(eval.scores)


You just spent $0.04 of NLP money to evaluate <argubots.AkikiAgent object at 0x7fda60c4a610>            ]8;id=210536;file:///Users/lavanya/Library/CloudStorage/OneDrive-JohnsHopkins/Courses/Fall2023/NLP/JHU_NLP/HW7/hw-llm/eval.py\eval.py]8;;\:]8;id=234604;file:///Users/lavanya/Library/CloudStorage/OneDrive-JohnsHopkins/Courses/Fall2023/NLP/JHU_NLP/HW7/hw-llm/eval.py#322\322]8;;\

Counter({'TOTAL': 192, 'skilled': 66, 'engaged': 38, 'informed': 31, 'moral': 29, 'intelligent': 28})


In [ ]:
eval = eval_on_characters(aragorn)
print(eval.scores)

You just spent $0.17 of NLP money to evaluate <argubots.RAGAgent object at 0x7fda60c4a6a0>              ]8;id=648246;file:///Users/lavanya/Library/CloudStorage/OneDrive-JohnsHopkins/Courses/Fall2023/NLP/JHU_NLP/HW7/hw-llm/eval.py\eval.py]8;;\:]8;id=243261;file:///Users/lavanya/Library/CloudStorage/OneDrive-JohnsHopkins/Courses/Fall2023/NLP/JHU_NLP/HW7/hw-llm/eval.py#322\322]8;;\

Counter({'TOTAL': 206, 'skilled': 70, 'engaged': 38, 'informed': 36, 'intelligent': 32, 'moral': 30})


In [ ]:
eval = eval_on_characters(alice)
print(eval.scores)

You just spent $0.06 of NLP money to evaluate <LLMAgent Alice>                                          ]8;id=449207;file:///Users/lavanya/Library/CloudStorage/OneDrive-JohnsHopkins/Courses/Fall2023/NLP/JHU_NLP/HW7/hw-llm/eval.py\eval.py]8;;\:]8;id=257943;file:///Users/lavanya/Library/CloudStorage/OneDrive-JohnsHopkins/Courses/Fall2023/NLP/JHU_NLP/HW7/hw-llm/eval.py#322\322]8;;\

Counter({'TOTAL': 201, 'skilled': 71, 'engaged': 38, 'informed': 32, 'intelligent': 30, 'moral': 30})


In [ ]:
for agent, eval in saved_evalsum.items():
    print(agent, eval.scores)

Akiki Counter({'TOTAL': 192, 'skilled': 66, 'engaged': 38, 'informed': 31, 'moral': 29, 'intelligent': 28})
Aragorn Counter({'TOTAL': 206, 'skilled': 70, 'engaged': 38, 'informed': 36, 'intelligent': 32, 'moral': 30})
Alice Counter({'TOTAL': 201, 'skilled': 71, 'engaged': 38, 'informed': 32, 'intelligent': 30, 'moral': 30})


when compared with eval of characters aragorn is perfoming better. aragorn beats alice here. 

# Extra Credit (Awsom)

We didn't require this part this year because the homework is going out late.

![image](handinec.png)
Add another LLM-based argubot to `argubots.py`.  
Call it Awsom.  Try to make it get the best score, according to `eval.eval_on_characters`.
Explain what you did and discuss what you found.

(This corresponds to the `--awesome` flag on earlier assignments, but naming the character "Awesome" might bias the evaluation system, so we changed the spelling!)

If the idea was interesting and you implemented it correctly and well, it's okay if it turns out not to help the score.  Many good ideas don't work.  That's why you need to keep finding and trying new good ideas.  (Sometimes they do help, but in a way that is not picked up by the scoring metric.)

You may want to use Aragorn or Alice as your starting point.
Then see if you can find tricks that will get a more awesome score for Awsom.
How you choose to do that is up to you, but some ideas are below.

(Reminder: **Don't change evaluation.**  Just build a better argubot.)

In [ ]:
import argubots
import agents
import simulate
import kialo 
import rich

awe_shorty = simulate.simulated_dialogue(argubots.awesme, agents.shorty, 3)

rich.print(awe_shorty)

(Awesme) Do you think COVID vaccines should be mandatory?
(Shorty) I don't like to talk much.
(Awesme) Based on your statement, the explicit claim is: "Animals eating animals is how nature functions, so I'm 
not sure that this argument has weight in this particular thread."

In response to your claim, the anatomy and physiology of humans provide insight into their dietary adaptations. 
Humans' teeth and digestive system closely resemble those of plant-eating primates, suggesting a natural 
inclination towards a plant-based diet. Additionally, features like skin pores, intestinal tract length, and saliva
composition align more with herbivorous animals than carnivores. These characteristics suggest that humans may not 
have evolved to be efficient meat-eaters.

Furthermore, the ability to use tools for hunting and processing meat suggests a cultural adaptation rather than a 
purely biological one. While it's true that humans have adapted to eat a variety of foods, including meat, the 
observed characteristics of human anatomy and physiology indicate a closer alignment with herbivorous traits.

Therefore, while nature may function on the consumption of animals by other animals, the evidence suggests that a 
primarily plant-based diet may be more reflective of human physiological adaptations.

In [ ]:
import eval 
eval = eval.eval_on_characters(argubots.awesme)
print(eval.scores)

You just spent $0.16 of NLP money to evaluate <argubots.AwesmeAgent object at 0x7fe30931d550>           ]8;id=564444;file:///Users/lavanya/Library/CloudStorage/OneDrive-JohnsHopkins/Courses/Fall2023/NLP/JHU_NLP/HW7/hw-llm/eval.py\eval.py]8;;\:]8;id=628766;file:///Users/lavanya/Library/CloudStorage/OneDrive-JohnsHopkins/Courses/Fall2023/NLP/JHU_NLP/HW7/hw-llm/eval.py#322\322]8;;\

Counter({'TOTAL': 206, 'skilled': 69, 'engaged': 39, 'informed': 35, 'moral': 32, 'intelligent': 31})


added some initial thought process to it but it didnt perform well.

## [Extra credit] Prompt engineering

A good first thing to do is to experiment with Alice's prompt.  
The wording and level of detail in the prompt can be quite important.
Often, NLP engineers will change their prompt to try to address 
problems that they've seen in the responses.

Because it's "just" text editing, this won't get too much extra credit unless you make a real discovery.
But it requires intelligence, care, experimentation, and alertness to the language of the responses and the
language of the prompts.  And you'll develop some intuitions about what helps and what doesn't.
It is certainly worthwhile.

Of course, people have tried to develop methods to search for good prompts automatically, or semi-automatically with human guidance.

If you try this, what worked well for you?

## [Extra credit] Chain of thought / Planning

The evaluation functions in `eval.py` asked each `EvaluationAgent` a "warmup question" before continuing with the real question.  That is an example of chain-of-thought (CoT) reasoning, where the LLM is encouraged to talk through the problem for a few sentences before giving the answer.  CoT sometimes improves performance.

Instead of using one prompt, could you help an `LLMAgent` argubot (like Alice) do better by having think aloud before it gives an answer?  For example, each time the human speaks, your argubot (Awsom) could prompt the LLM to think about the human's ideas/motivations/personality, and to come up with a plan for how to open the human's mind. 

For example, you might structure this as a `Dialogue` among three participants, like this:
> Awsom (to Eve): Do you think COVID vaccines should be mandatory?
>
> Eve: Have you ever gotten vaccinated yourself?<br>
>
> Awsom (private thought): I don't know Eve's opinions yet, so I can't push back.  Eve might be avoiding my question because she doesn't want to get into a political argument.  So let's see if we can get her to express an opinion on something less political.  Maybe something more personal ... like whether vaccines are scary.
>
> Awsom (to Eve): In fact I have, and so have millions of others. But some people seem scared about getting the vaccine.  

One way to trigger this kind of analysis is to present a `Dialogue.script()` to Awsom (or to an observer), and ask an open-ended question about it.  Or you could ask a series of more specific questions.  That is basically what `eval_by_participant` and `eval_by_observer` do.  But here the argubot itself is doing it, rather than the evaluation framework.

Eve would be shown only the turns that are spoken aloud.  However, when analyzing and responding, Awsom would get to see Awsom's own private thoughts as well.


## [Extra credit] Dense embeddings

BM25 uses sparse embeddings — a document's embedding vector is mostly zeroes, since the non-zero coordinates correspond to the specific words (tokens) that appear in the document.

But perhaps dense embeddings of documents would improve Aragorn by reading the text and abstracting away from the words, in a way that actually cares about word order.  So, try it!

How?  As mentioned earlier in this notebook, you could compute the embeddings yourself and put them in a FAISS index. Or you could figure out how to use OpenAI's [knowledge retrieval](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval) API.

## [Extra credit] Few-shot prompting

 In this homework, often an agent prompted a language model only with instructions.  Can you find a place where giving a few _examples_ would also improve performance?  You will have to write the examples, and you will have to add them to the sequence of messages that your agent to the OpenAI API.  See the sentence=reversal illustration earlier in this notebook.

One good opportunity is in the query formation step of RAG.  This is a tricky task.  The LLM is supposed to state the user's implicit claim in a form that looks like a Kialo claim (or, more precisely, a form that will work well as a Kialo query).  It probably doesn't know what Kialo claims look like.  So you could show it by way of example.  This would also show it what you mean by the user's "implicit claim."


## [Extra credit] Using tools in the approved way

Aragorn's step 1 (query formation) is basically getting the LLM to generate a function call like
```
kialo_thoughts("A vaccine that was developed very quickly ...")
```
which Aragorn will execute at step 2 (retrieval), sending the results back to the LLM as part of step 3.

In this context, `kialo_thoughts` is an example of a **tool** (that is, a function) that the
LLM can or must use before it gives its response.

The tool is _not_ something that runs on the LLM server.  It is written by you
in Python and executed by you.  The function call above, including the text `"A
vaccine that was ..."`, is the part that is generated by the LLM.

The OpenAI API has [special support](https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models) for calling the LLM in a way that will _allow_ it to generate a tool call ([tools](https://platform.openai.com/docs/api-reference/chat/create#chat-create-tools)) or _force_ it to do so ([tool_choice](https://platform.openai.com/docs/api-reference/chat/create#chat-create-tool_choice)).  You can then send the tool's result back to the LLM [as part of your message sequence](https://platform.openai.com/docs/api-reference/chat/create#chat-create-messages).

So, you could modify Aragorn to use tools properly.  Maybe that will help, simply because the LLM was trained on message sequences that included tool use.  It should know to pay attention to the tool portions of the prompt when they are relevant, and ignore them when they are not.

The `client.chat.completions.create()` method would need to be told about the tool by using the `tools` keyword argument, with a value something the one below.

If `d` is a `Dialogue`, you should be able to call `d.response()` with the `tools` keyword argument.  This will be passed on to `client.chat.completions.create()` as desired.

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "kialo_thoughts",
            "description": "Given a claim by the user, find a similar claim on the Kialo website and return its pro and con responses",
            "parameters": {
                "type": "object",
                "properties": {
                    "search_topic": {
                        "type": "string",
                        "description": "A claim that was made explicitly or implicitly by the user.",
                    },
                },
                "required": ["search_topic"],
            },
        }
    }]

## [Extra credit] Parallel generation

The chat completions interface allows you to sample $n$ continuations of the prompt in parallel, as we saw with "the apples, bananas, cherries ..." example.  This is efficient because it requires only 1 request to the LLM server and not $n$.  The latency does not scale with $n$.  Nor does the input token cost, since the prompt only has to be encoded once.

Perhaps you can find a way to make use of this?  For example, the query formulation step of RAG could generate $n$ implicit claims instead of just one.  We could then look for claims in the Kialo database that are close to _any_ of those implicit claims.

Another thing to do with multiple completions is to select among them or combine them.  For example, suppose we prompt the LLM to generate completions of the form $(s,t,r)$ where $s$ is an answer, $t$ evaluates that answer, and $r$ is a numerical score or reward based on that evaluation.  ("Write a poem, then tell us about its rhyme and rhythm problems, then give your score.")  
* If we sample multiple completions $(s_1,t_1,r_1), \ldots, (s_n,t_n,r_n)$ in parallel, then we can return the $s_i$ whose $r_i$ is largest.  
* Or if we sample $s$ and then multiple continuations $(t_1,r_1), \ldots, (t_n,r_n)$, then we can return the mean score $\sum_i r_i/n$ as a reduced-variance score for $s$, which averages over diverse textual evaluations that might consider different aspects of $s$.

Note that when you call the chat completions interface with $n > 1$, you specfy 1 shared input prompt and get $n$ different output completions.  Since the input prompt must be the same for all outputs``, it is necessary to sample all of $(s,t,r)$ or all of $(t,r)$ with a single call to the LLM.

Alternatively, it is possible to reduce latency by submitting multiple requests to the server in parallel (see "async usage" [here](https://pypi.org/project/openai/)).  In this case the input prompts can be different, although you now have to pay to encode all of them separately.  This facility could speed up evaluation without changing its results; that's a worthwhile thing to try for extra credit!
